In [1]:
import win32api, win32con, win32gui
import ctypes
import pygetwindow as gw
from PIL import Image, ImageGrab
import cv2
import numpy as np
import pandas as pd
from functions import *

In [12]:
# list all open windows
# def winEnumHandler( hwnd, ctx ):
#     if win32gui.IsWindowVisible( hwnd ):
#         print (hex(hwnd), win32gui.GetWindowText( hwnd ))

# win32gui.EnumWindows( winEnumHandler, None )

In [14]:
window_size, window_top_left = obtain_window_info()
box_cord = (round(window_top_left[0]),
            round(window_top_left[1]),
            round(window_top_left[0] + window_size[0]),
            round(window_top_left[1] + window_size[1])
           )
im = cv2.imread("./game_snippets/snippet_4.png")

In [15]:
boxes_top_index = 127 # pixel 128
boxes_left_index = 19 # pixel 20
boxes_right_index = im.shape[1] - 20
boxes_bottom_index = im.shape[0] - 19
np_boxes = im[boxes_top_index : boxes_bottom_index, boxes_left_index:boxes_right_index, :]
height = int((np_boxes.shape[0] + 1)/20)
width = int((np_boxes.shape[1] + 1)/20)

In [16]:
# calculate the average pixel values for each box
box_middle_mean = np.zeros([height, width, 3])
box_top_edge_mean = np.zeros([height, width, 3])
for i in range(height):
    for j in range(width):
        ### pixel positions for each box
        top = 20 * i
        bottom = top + 19
        left = 20 * j
        right = left + 19

        for k in range(3):
            box_top_edge_mean[i,j, k] = np.mean(np_boxes[top, left:right, k])

        ### trim the edge pixels
        top = top + 2
        bottom = bottom - 2
        left = left + 2
        right = right - 2

        for k in range(3):
            box_middle_mean[i,j, k] = np.mean(np_boxes[top:bottom, left:right, k])

In [20]:
middle_pixel_mean = np.array([[1,210,138,138], [2,103,162,103],
                             [3,107,107,220], [4,167,117,117],
                             [5,96,96,160], [6,159,159,94],
                              [7,132,132,132],
                             [9,143,143,173], [10,74,74,143],
                             [11,94,94,94], [12,7,7,122], [13,195,195,195]])

top_edge_pixel_mean = np.array([[0,130,130,130], [13,253,253,253]])

In [27]:
box_number = np.ones([height, width])
box_number = box_number * 8
for i in range(height):
    for j in range(width):
        for k in range(middle_pixel_mean.shape[0]):     
            if (box_middle_mean[i,j,0] > middle_pixel_mean[k, 1] - 5) & \
            (box_middle_mean[i,j,0] < middle_pixel_mean[k, 1] + 5) & \
            (box_middle_mean[i,j,1] > middle_pixel_mean[k, 2] - 5) & \
            (box_middle_mean[i,j,1] < middle_pixel_mean[k, 2] + 5) & \
            (box_middle_mean[i,j,2] > middle_pixel_mean[k, 3] - 5) & \
            (box_middle_mean[i,j,2] < middle_pixel_mean[k, 3] + 5):
                box_number[i,j] = middle_pixel_mean[k, 0]
                
for i in range(height):
    for j in range(width):                                
        if box_number[i,j] == 13:
            for l in range(top_edge_pixel_mean.shape[0]):
                if (box_top_edge_mean[i,j,0] > top_edge_pixel_mean[l, 1] - 20) & \
                (box_top_edge_mean[i,j,0] < top_edge_pixel_mean[l, 1] + 20) & \
                (box_top_edge_mean[i,j,1] > top_edge_pixel_mean[l, 2] - 20) & \
                (box_top_edge_mean[i,j,1] < top_edge_pixel_mean[l, 2] + 20) & \
                (box_top_edge_mean[i,j,2] > top_edge_pixel_mean[l, 3] - 20) & \
                (box_top_edge_mean[i,j,2] < top_edge_pixel_mean[l, 3] + 20):
                    box_number[i,j] = top_edge_pixel_mean[l, 0]

In [28]:
box_number

array([[11., 13., 13., 13., 13., 13., 13., 13., 13., 13., 11., 13., 11.,
        11., 13., 11., 11., 13., 13., 11., 11., 13., 13., 13., 11., 13.,
        13., 13., 13., 13.],
       [13., 11., 13., 13., 13., 11., 13., 13., 11., 13., 13., 12.,  4.,
         9.,  9.,  9.,  6., 11., 13.,  9.,  9.,  9.,  9., 13., 13., 13.,
        13., 13., 11., 11.],
       [13., 13., 13., 11., 11., 11., 13., 11., 13., 13., 11., 13.,  2.,
         2.,  3.,  3.,  9.,  9.,  9.,  9.,  4.,  3.,  3., 11., 11., 13.,
        13., 13., 13., 13.],
       [13., 11., 13., 13., 13., 11., 11., 11., 13., 13., 13., 13.,  1.,
         0.,  0.,  1.,  2.,  3.,  3.,  2.,  1.,  1.,  2., 13., 11., 11.,
        11., 13., 13., 11.],
       [13., 13., 13., 13., 13., 13., 13., 13., 11., 13., 13., 11.,  2.,
         0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  2.,  9., 13., 13., 13.,
        13., 13., 13., 11.],
       [13., 13., 11., 13., 11., 11., 13., 13., 13., 13., 13., 11.,  4.,
         3.,  3.,  9.,  2.,  1.,  0.,  0.,  0.,  2.,

In [15]:
# calculate the average pixel values for each box

# calculate the average pixel values for each box
box_mean = np.zeros([height, width, 3])
box_first_row_mean = np.zeros([height, width, 3])
for i in range(height):
    for j in range(width):
        ### pixel positions for each box
        top = 20 * i
        bottom = top + 19
        left = 20 * j
        right = left + 19

        for k in range(3):
            box_first_row_mean[i,j, k] = np.mean(np_boxes[top, left:right, k])

In [24]:
box_top_edge_mean[:,:,2]

array([[130.78947368, 253.05263158, 253.36842105, 253.36842105,
        253.36842105, 253.36842105, 253.36842105, 253.36842105,
        253.36842105, 253.36842105, 131.10526316, 253.05263158,
        131.10526316, 130.78947368, 253.05263158, 131.10526316,
        130.78947368, 253.05263158, 253.36842105, 131.10526316,
        130.78947368, 253.05263158, 253.36842105, 253.36842105,
        131.10526316, 253.05263158, 253.36842105, 253.36842105,
        253.36842105, 253.36842105],
       [253.10526316, 131.10526316, 253.10526316, 253.42105263,
        253.42105263, 131.10526316, 253.10526316, 253.42105263,
        131.10526316, 253.10526316, 253.42105263, 133.89473684,
        130.78947368, 253.10526316, 253.42105263, 253.42105263,
        131.10526316, 130.78947368, 253.10526316, 253.42105263,
        253.42105263, 253.42105263, 253.42105263, 253.42105263,
        253.42105263, 253.42105263, 253.42105263, 253.42105263,
        131.10526316, 130.78947368],
       [253.15789474, 253.4736

In [ ]:
# first identify the size of the map (height x width)
boxes_top_index = 127 # pixel 128
boxes_left_index = 19 # pixel 20
boxes_right_index = im.shape[1] - 20
boxes_bottom_index = im.shape[0] - 19
np_boxes = im[boxes_top_index : boxes_bottom_index, boxes_left_index:boxes_right_index, :]
height = int((np_boxes.shape[0] + 1)/20)
width = int((np_boxes.shape[1] + 1)/20)